# Imports

In [30]:
%pylab inline
import pandas as pd
import seaborn as sns


from scipy.spatial.distance import euclidean
from scipy.spatial import distance_matrix
from scipy.stats import binom_test 
import scipy.stats

import collections
from itertools import combinations
import string

%load_ext autoreload
%autoreload 2

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import Co_Evo_Func as cef

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Analysis

## Preproccessing

In [ ]:
counts = pd.read_excel("raw_data/counts_uni.xlsx", index_col=0)

counts['Generation'] = counts['transfer']*10.5  # translate transfers to generations
species = counts.loc[:, 'Ea':'IN72'].columns # list of species in the experiment
counts['total'] = counts[species].sum(axis = 1) # total counts for each row
counts = counts[counts['total']>0] # Remove rows with no counts
counts['present'] = counts[species].apply(lambda x:x>0, axis = 1).apply(lambda x: list(species[x.values]), axis=1) # which species are present at each count
counts = counts[counts['sample_kind'].isin(['Pair', 'Trio'])].reset_index(drop = True) # leave only pairs and trios
counts = counts[~counts['transfer'].isin([13, 15])].reset_index(drop = True) # Two transfers were we sampled only a small number of communities


Samples that did not coexsist for at least 70 generations are taken out of the analysis

In [32]:
coexistence = lambda x : True if all([sp in x['present'] for sp in x['sample'].split('_')]) else False
counts['coexistence'] = counts.apply(coexistence, axis = 1) # are all species that should be present present?
excluded = lambda x:[sp for sp in x['sample'].split('_') if sp not in x['present']][0] if len([sp for sp in x['sample'].split('_') if sp not in x['present']]) > 0 else 'non'
counts['excluded'] = counts.apply(excluded, axis =1) # Which species isn't present?

last_coex = lambda x: max(x['transfer'][x['coexistence']]) 
lc = counts.groupby(['sample_kind','sample','ident'])[['transfer', 'coexistence']].apply(last_coex) # what is the last timepoint that all species were present?
lc = pd.DataFrame(lc).reset_index()
lc = lc.rename(columns={0: 'last_coex'})
lc['excluded'] = lc.apply(lambda x:counts['excluded'][counts['ident']==x['ident']][counts['transfer']==38].values, axis = 1)


counts = counts[~counts['ident'].isin(lc['ident'][lc['last_coex']<7])].reset_index(drop = True) # remove ecologically non-coexsiting communities
counts = counts[counts['sample']!='Pa_Fj'].reset_index(drop = True) # Only in one well, at one time point a single colony of Fj apeared after generation ~70

Remove contaminated samples from the analysis

In [ ]:
spot_contaminations = lambda x : False if all([sp in x['sample'].split('_') for sp in x['present']]) else True
counts['cont'] = counts.apply(spot_contaminations, axis = 1) # when a species which souldn't be present is present, set cont as True
first_cont = lambda x: min(x['transfer'][x['cont']].values) if (len(x['transfer'][x['cont']]) != 0) else 38 
fcont = counts[counts['sample_kind'].isin(['Pair', 'Trio'])].groupby(['ident'])[['transfer', 'cont']].apply(first_cont) # If cont == True where did it apear first
fcont = pd.DataFrame(fcont).reset_index()
for t, ide in zip(fcont[fcont[0]!=38][0], fcont[fcont[0]!=38]['ident']):
    # all datapoints after contamination appeared are considered contaminated
    counts['cont'][counts['ident']==ide][counts['transfer'] > t] = True
contaminated = counts['ident'][counts['cont']].unique()
with open("proccesed_data/contamination.txt", "w") as output:
    output.write(str(contaminated))
    
counts = counts[~counts['cont']].reset_index(drop = True)

In [7]:
 #### cf is noramlized version of counts ####
cf = counts.copy() 
cf[species] = cf[species].apply(lambda x:x/cf['total'])
#counts_frac['detection_limit'] = 1/counts_frac['total']

only_pairs = cf['present'].apply(lambda x:len(x))==2 #return only columns with 2 species present
first_species = lambda x:x.split('_')[0] # return the first species
# calculate the std from binome distribution
std_binom = lambda x: np.sqrt(x[first_species(x['sample'])]*(1-x[first_species(x['sample'])])/(x['total']+1)) 
cf['std'] = cf[only_pairs].apply(std_binom, axis = 1)


#### cf_psu is noramlized version of counts with added pseudocounts ####
cf_psu = counts.copy() 
cf_psu = cf_psu.apply(cef.add_pseudocounts, axis = 1)
cf_psu['total'] = cf_psu[species].sum(axis = 1)

cf_psu[species] = cf_psu[species].apply(lambda x:x/cf_psu['total'])
only_pairs = cf_psu['present'].apply(lambda x:len(x))==2 #return only columns with 2 species present
first_species = lambda x:x.split('_')[0] # return the first species
# calculate the std from binome distribution
std_binom = lambda x: np.sqrt(x[first_species(x['sample'])]*(1-x[first_species(x['sample'])])/(x['total']+1)) 
cf_psu['std'] = cf_psu[only_pairs].apply(std_binom, axis = 1)
####

### We define change as the Euclidean distance between a sample at two timepoints, normalized to the maximum possible change
cf['change'] = cf.groupby('ident')[species].apply(cef.euc_change)# quantify the distance of each timepoint from the last one
cf['change'] = cf.apply(lambda x:x['change']/sqrt(len(x['sample'].split('_'))), axis = 1) # the maximum possible change = sqrt(n)

### dist from ee is the change since generation 70
cf['dist_from_ee'] = cf.apply(lambda x: euclidean(x[species], cf[species][cf['transfer']==7][cf['ident'] == x['ident']]) if len(cf[species][cf['transfer']==7][cf['ident'] == x['ident']])!= 0 else NaN, axis = 1)
cf['dist_from_ee'] = cf.apply(lambda x:x['dist_from_ee']/sqrt(len(x['sample'].split('_'))), axis = 1)
### distance from centroid is the distance of each replicate from the centroid across replicates
# cf['dist_from_cent'] = cf.apply(lambda x:euclidean(x[species], cf[species][cf['sample']==x['sample']][cf['transfer']==x['transfer']].mean()), axis =1)

### OD Data ###
OD = pd.read_excel("proccesed_data/OD_proccesed.xlsx") # OD table

### frac_od gives the abundances as fractional od (od X fraction)
frac_od = pd.merge(cf, OD[['ident', 'transfer', 'OD', 'smoothed']], on=['ident', 'transfer'])
frac_od[species] = frac_od[species].apply(lambda x:x*frac_od['smoothed'])

cf['Gen_jit'] = cf['Generation']+ random.normal(0, 2, len(cf)) #jittered generation for some plots
cf['Gen_jit'][cf['transfer']==0] = 0 #don't jitter t0

# cf.to_excel('proccesed_data/cf.xlsx', index=False)
# cf_psu.to_excel('proccesed_data/cf_psu.xlsx', index=False)
# frac_od.to_excel('proccesed_data/frac_od.xlsx', index=False)
# counts.to_excel('proccesed_data/counts_proccesed.xlsx', index = False)

# cf = pd.read_excel('proccesed_data/cf.xlsx') # fractionized count table
# cf_psu = pd.read_excel('proccesed_data/cf_psu.xlsx') # fractionized count table with pseudocounts
# frac_od = pd.read_excel('proccesed_data/frac_od.xlsx')

# OD = pd.read_excel("raw_data/OD_proccesed.xlsx") # OD table

pairs = cf['sample'][cf['sample_kind']=='Pair'].unique() # a list of pairs in the experiment
trios = cf['sample'][cf['sample_kind']=='Trio'].unique() # a list of trios in the experiment
species = cf.loc[:, 'Ea':'IN72'].columns # list of species in the experiment
transfers = cf['transfer'].unique()
gens = cf['Generation'].unique()
cf['total'][cf.transfer == 0] = cf['total'][cf.transfer == 0] + 10
idents = cf['ident'].unique()


C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning:

C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning:

C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning:

C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning:

C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning:

...

C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning:

C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning:

C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning:

C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: UserWarning:

C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\friedman-lab\Anaconda3\lib\site-packages\ipykernel_launcher.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Aggregated tables

In [9]:
#### group cf by sample for each t, avrage across replicates: ###
bysamp = cf.groupby(['sample', 'transfer', 'sample_kind', 'Generation'], as_index=False).mean() 
bysamp = bysamp.drop(['cont','std'], axis = 1) # drop some irelevent coloumns
bysamp['size'] = cf.groupby(['sample', 'transfer'], as_index=False).aggregate(np.size)['sample_kind'] 
# calculate the avarage distance from mediod for each
bysamp['dist_from_med'] = cf.groupby(['sample', 'transfer', 'sample_kind', 'Generation'])[species].apply(cef.dist_from_med).values 
bysamp['dist_from_med'] = bysamp.apply(lambda x:x['dist_from_med']/sqrt(len(x['sample'].split('_'))), axis = 1)
# bysamp['dist_from_cent_sd'] = cf[['sample', 'transfer', 'dist_from_cent']].groupby(['sample', 'transfer']).std().values

### dist_from_cent, and the z_score are ment to give a statistical 

### by_ide is aggregated for each replicate across time
by_ide = cf.groupby(by = ['sample', 'ident', 'sample_kind'], as_index=False).mean()[['sample', 'ident', 'sample_kind']]
by_ide['change'] = cf.groupby('ident', as_index=False)['dist_from_ee'].apply(lambda x:x[x.index[len(x)-1]])
# by_ide['z_score'] = by_ide.apply(lambda x:cef.zscore(x['change'], 
#                              bysamp['dist_from_cent'][bysamp['transfer']==7][bysamp['sample']==x['sample']].values, 
#                              bysamp['dist_from_cent_sd'][bysamp['transfer']==7][bysamp['sample']==x['sample']].values), axis = 1)
# by_ide['z_score'] = by_ide['z_score'].apply(lambda x:x[0] if x else NaN)


sum_data = by_ide.groupby(['sample','sample_kind'], as_index=False).mean()
sum_data['beta_div'] = bysamp[bysamp['transfer']==38].merge(sum_data, on = 'sample')['dist_from_med']
sum_data['beta_div_norm'] = log(sum_data['beta_div']/bysamp[bysamp['transfer']==7].merge(sum_data, on = 'sample')['dist_from_med'])

# bysamp.to_excel('proccesed_data/by_samp.xlsx')
# by_ide.to_excel('proccesed_data/by_ide.xlsx')
# sum_data.to_excel('proccesed_data/sum_data.xlsx')

C:\Users\friedman-lab\Google Drive\shared\lab_members\Nittay_Meroz\manuscripts\predictability of community structure\github_resp\Co_Evo_Func.py:83: RuntimeWarning: Mean of empty slice.
  return dm[minimum][np.arange(0, len(dm))!=minimum].mean()/np.sqrt(n)
C:\Users\friedman-lab\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Repeatability

We measure the varibility between replicate communities as the avrage distance from medoid. We compare these to the null model that species fractions are derived from a uniform distribution. 

In [19]:
### produce a random version of cf, where species fractions are derived from uniform distribution.
rand_cf = cf.apply(lambda x: cef.randomize_counts(x), axis = 1)
rand_grouped = rand_cf.groupby(['sample', 'Generation'])[species].apply(cef.dist_from_med).reset_index()
rand_grouped['var'] = rand_grouped.apply(lambda x:x[0]/sqrt(len(x['sample'].split('_'))), axis = 1)
#rand_cf.to_excel('proccesed_data/rand_cf.xlsx')

### test the significance with Mann-Whitney u-test
from scipy.stats import mannwhitneyu

like_rand = pd.DataFrame(columns=['Generation', 'U', 'P'])
like_rand['Generation'] = cf['Generation'].unique()
like_rand['U'] = like_rand['Generation'].apply(lambda x:mannwhitneyu(bysamp[bysamp['Generation']==x]['dist_from_med'].values,
                                                                     rand_grouped.reset_index()[rand_grouped.reset_index()['Generation']==x][0].values,
                                                                      alternative = 'less')[0])
like_rand['P'] = like_rand['Generation'].apply(lambda x:mannwhitneyu(bysamp[bysamp['Generation']==x]['dist_from_med'].values,
                                                                     rand_grouped.reset_index()[rand_grouped.reset_index()['Generation']==x][0].values,
                                                                       alternative = 'less')[1])
like_rand

C:\Users\friedman-lab\Google Drive\shared\lab_members\Nittay_Meroz\manuscripts\predictability of community structure\github_resp\Co_Evo_Func.py:83: RuntimeWarning: Mean of empty slice.
  return dm[minimum][np.arange(0, len(dm))!=minimum].mean()/np.sqrt(n)
C:\Users\friedman-lab\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,Generation,U,P
0,0.0,170.0,2.817583e-31
1,21.0,485.0,1.546091e-23
2,52.5,415.0,1.828645e-24
3,73.5,384.0,7.005597e-25
4,105.0,643.0,1.629033e-21
5,147.0,1381.0,2.720169e-10
6,199.5,679.0,4.562219e-21
7,315.0,781.0,7.924354e-20
8,399.0,944.0,6.255399e-18


In order to quantify the qualitative repeatability of different replicate communities we first identified which species was the maximally increasing member at each replicate, that is, which species had increased its abundance by the largest factor between generation 70 and 400. Then, we quantified the frequency of the replicates that had the same maximally increasing member for each community. This measure always produces a value between 1 and 1/n where n is the number of species in the community. For this we used the count table with added pseudocounts.

In [20]:
end = cf_psu['transfer']==38
ee = cf_psu['transfer']==7
rep_all = by_ide.copy()

# Get the fractional OD of the species at ee and end:
rep_all['fractions_od_t38'] = rep_all.apply(lambda x:cef.get_fractional_OD(cf_psu, OD, x['sample'].split('_'), x['ident'], 38), axis = 1)
rep_all['fractions_od_t7'] = rep_all.apply(lambda x:cef.get_fractional_OD(cf_psu, OD, x['sample'].split('_'), x['ident'], 7), axis = 1)
# Remove missing data:
rep_all = rep_all[~rep_all['fractions_od_t38'].apply(lambda x:size(x) ==0)][~rep_all['fractions_od_t7'].apply(lambda x:size(x) ==0)]
# Calutlate the fold increase in fractional OD:
rep_all['fold_increase'] = rep_all['fractions_od_t38']/rep_all['fractions_od_t7']
# Find the maximum increasing memmber:
rep_all['max'] = rep_all['fold_increase'].apply(argmax)

# All increasing values for pairs and for trios:
pairs_inc, trios_inc = concatenate(rep_all['fold_increase'][rep_all['sample_kind']=='Pair'].values),concatenate(rep_all['fold_increase'][rep_all['sample_kind']=='Trio'].values)


# All increasing values for each species in pairs and in trios:
pairs_bysp, trios_bysp = {},{}
for sp in species:
    get_inc_values = lambda x: x['fold_increase'][where(array(x['sample'].split('_'))==sp)[0][0]] if sp in x['sample'].split('_') else NaN
    pairs_bysp[sp] = rep_all[rep_all['sample_kind']=='Pair'].apply(get_inc_values, axis = 1).dropna().values
    trios_bysp[sp] = rep_all[rep_all['sample_kind']=='Trio'].apply(get_inc_values, axis = 1).dropna().values

# summarize results:
rep_all_sum = rep_all.groupby(['sample','sample_kind', 'max'])['max'].count().groupby(['sample', 'sample_kind']).apply(cef.pmax).reset_index()
rep_all_sum['most_freq'] = rep_all.groupby('sample')['max'].apply(cef.most_frequent).values
rep_all_sum['size'] = rep_all.groupby('sample')['max'].size().values


We checked the distribution of the repeatability scores against the null hypothesis hat the factor by which a species’ abundance increases during evolution is independent of the species or the community. We iterate over data that is randomly produced under this null hypothesis 2000 times.

In [22]:
# Iterate over the two H0. One randomized across all, and one randomized over strains
rep_all_sum = rep_all_sum[rep_all_sum['size']>2]
ite= 2000
si = rep_all_sum.shape[0]
pairs_random_dist, trios_random_dist = [],[]

temp = rep_all[rep_all['sample'].isin(rep_all_sum['sample'])].copy()

for i in range(ite):
    temp['fold_increase'] = temp.apply(lambda x:cef.randomize_incs(x['sample'], pairs_inc, 2) if x['sample_kind'] == 'Pair' else cef.randomize_incs(x['sample'], trios_inc, 3), axis = 1)
    pairs_random_dist.append(cef.summarize_repeatability(temp[temp['sample_kind']=='Pair']))
    trios_random_dist.append(cef.summarize_repeatability(temp[temp['sample_kind']=='Trio']))
    
# savetxt('proccesed_data/permutations/pairs_random_dist.txt', pairs_random_dist)
# savetxt('proccesed_data/permutations/pairs_bysp_dist.txt', pairs_bysp_dist) 
# savetxt('proccesed_data/permutations/trios_random_dist.txt', trios_random_dist)
# savetxt('proccesed_data/permutations/trios_bysp_dist.txt', trios_bysp_dist) 

C:\Users\friedman-lab\Google Drive\shared\lab_members\Nittay_Meroz\manuscripts\predictability of community structure\github_resp\Co_Evo_Func.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['max'] = df['fold_increase'].apply(np.argmax)


In [28]:
# pairs_random_dist =  np.loadtxt('proccesed_data/permutations/pairs_random_dist.txt')
# pairs_bysp_dist   =  np.loadtxt(  'proccesed_data/permutations/pairs_bysp_dist.txt')
# trios_random_dist =  np.loadtxt('proccesed_data/permutations/trios_random_dist.txt')
# trios_bysp_dist   =  np.loadtxt(  'proccesed_data/permutations/trios_bysp_dist.txt')
rep_all_sum = rep_all_sum[rep_all_sum['size']>2]

ite= 2000

mean_data_pairs = rep_all_sum[rep_all_sum['sample_kind']=='Pair']['max'].mean()
mean_data_trios = rep_all_sum[rep_all_sum['sample_kind']=='Trio']['max'].mean()
pairs_rand_noiter = array(pairs_random_dist).mean(axis = 1)[array(pairs_random_dist).mean(axis = 1)>=mean_data_pairs].shape[0]
trios_rand_noiter = array(trios_random_dist).mean(axis = 1)[array(trios_random_dist).mean(axis = 1)>=mean_data_trios].shape[0]

print('The p-value gives the frequency of null-model iterations that give a mean equal or higher then the experimental mean\n')
print(pd.DataFrame(data = array([[mean_data_pairs, mean_data_trios],
                           [pairs_rand_noiter/ite, trios_rand_noiter/ite]]),
             columns=['Pairs', 'Trios'], 
             index = ['mean repeatability score', 'Fully shuffled p-value']))

The p-value gives the frequency of null-model iterations that give a mean equal or higher then the experimental mean

                             Pairs     Trios
mean repeatability score  0.852929  0.857348
Fully shuffled p-value    0.000000  0.000000


## Predictions

We used the formerly established methods for predicting the composition of trios from the composition of pairs that was developed by Abreu et al. In this approach the fraction of a species when grown in a multispecies community is predicted as the weighted geometric mean of the fraction of the species in all pairwise cultures

When prediciting by monocultures, we predict the fraction of a species in a community as its relatvie productivity in monoculuture:
$$f_{ic_{t}} = \frac{Y_{i_{t}}}{\sum{Y_{j_{t}}}}$$
where $f_{ic_{t}}$ is the fraction of species i at time t within community c;
$Y_{i_{t}}$ is the OD od species i at time t when grown in monoculutre; and $\sum{Y_{j_{t}}}$ is the sum OD of all of the species when grown in monoculutre

When predicting by pairs, we predict the fraction of a species in a multispecies community as its weighted geometric mean in pairs:
$$f_{1_{t}} = (f^{w_2}_{12}f^{w_3}_{13})^{\frac{1}{w_2w_3}} $$

where 
$$w_{2} = \sqrt{f_{21}f_{23}}, w_{3} = \sqrt{f_{31}f_{32}}$$

These fractions are normalizes by $f^*_{1} = \frac{f_1}{f_1+f_2+f_3}$

In [29]:


prediction_table = pd.DataFrame(columns=['sample', 'Generation', 'observation', 'pair_prediction', 'mono_prediction'])

prediction_table['sample'] = tile(trios, len(gens[gens!=0]))
prediction_table['Generation'] = repeat(gens[gens!=0], len(trios))

# actual observations:
prediction_table['observation'] = prediction_table.apply(lambda x: bysamp[x['sample'].split('_')][bysamp['sample']==x['sample']][bysamp['Generation']==x['Generation']].values.squeeze(), axis = 1)

# predicted by pairs:
prediction_table['pair_prediction'] = prediction_table.apply(lambda x: cef.pair_trio_prediction(x['sample'], bysamp[bysamp['Generation']==x['Generation']]).values.squeeze(), axis = 1)
# predicted by monoculture:
prediction_table['mono_prediction'] = prediction_table.apply(lambda x:[OD['OD'][OD['sample']==sp][OD['Generation']==x['Generation']].mean() for sp in x['sample'].split('_')]/sum([OD['OD'][OD['sample']==sp][OD['Generation']==x['Generation']].mean() for sp in x['sample'].split('_')]).mean(), axis = 1)
#predicted by pairs at generation 70:

prediction_table['pred_by70'] = prediction_table.apply(lambda x: cef.pair_trio_prediction(x['sample'], bysamp[bysamp['Generation']==7*10.5]).values.squeeze(), axis = 1)

# removing NaN
prediction_table = prediction_table[prediction_table['pair_prediction'].apply(lambda x:x[0] >-1)]
prediction_table = prediction_table[prediction_table['observation'].apply(lambda x:len(x)>0)]

we measure the accuracy of the error as the Euclidean distance between the observation and the prediction normalized to the maximum distance between two communities with the same n species - $\sqrt(n)$

In [31]:
prediction_table['By_pairs'] = prediction_table.apply(lambda x: euclidean(x['pair_prediction'], x['observation'])/sqrt(3), axis = 1)
prediction_table['By_pairs70'] = prediction_table.apply(lambda x: euclidean(x['pred_by70'], x['observation'])/sqrt(3), axis = 1)
prediction_table['By_mono'] = prediction_table.apply(lambda x: euclidean(x['mono_prediction'], x['observation'])/sqrt(3), axis = 1)

#the uniformed guess is that the species are at equal abundances
prediction_table['Uninformed'] = prediction_table.apply(lambda x: euclidean([1/3, 1/3, 1/3], x['observation'])/sqrt(3), axis = 1)


prediction_sum = prediction_table[["Generation", 'By_pairs', 'By_mono', 'Uninformed','By_pairs70']].groupby('Generation')